# Requirements

In [1]:
import itertools
import math
import sys

We define a simple function to test whether a given number is a prime.

In [2]:
def is_prime(n):
    '''verify whether a given integer is a prime
    
    Paraemeters
    -----------
    n : int
        value to check
        
    Returns
    -------
    bool
        True if the number is prime, False otherwise
    '''
    if n < 2:
        return False
    for divisor in range(2, int(round(math.sqrt(n), 0)) + 1):
        if n % divisor == 0:
            return False
    return True

We test it on a number of values.

In [3]:
for n in range(50):
    if is_prime(n):
        print(f'{n} is prime')

2 is prime
3 is prime
5 is prime
7 is prime
11 is prime
13 is prime
17 is prime
19 is prime
23 is prime
29 is prime
31 is prime
37 is prime
41 is prime
43 is prime
47 is prime


Now we want something to give us an indetermined number of primes.  We do not want to specify
* the maximum value of the value returned, or
* the number of values to be returned.

There are at least two approaches, one using object orientation by implementing an iterator class, the other implementing a generator.

# Iterator

An iterator class has to implement two methods:
1. `__iter__`: this returns an instance of the class taht is initialized;
1. `__next__`: this method will return the consecutive values for this iterator, it computes the next value to return.

In [4]:
class Primes:
    '''Iterator that returns all prime numbers (if you have enough patience, that is).
    
    Example
    -------
    for n in Primes():
        print(n)
    '''
    
    @staticmethod
    def _is_prime(n):
        '''verify whether a given integer is a prime
    
        Paraemeters
        -----------
        n : int
            value to check

        Returns
        -------
        bool
            True if the number is prime, False otherwise
        '''
        if n < 2:
            return False
        for divisor in range(2, int(round(math.sqrt(n), 0)) + 1):
            if n % divisor == 0:
                return False
        return True

    def __iter__(self):
        '''initializes iterator
        
        Returns
        -------
        Primes
            initialized iterator
        '''
        self._n = 2
        return self
    
    def __next__(self):
        '''next value of the iterator
        
        Returns
        -------
        int
            next prime number in the sequence
        '''
        if self._n == 2:
            self._n = 3
            return 2
        result = self._n
        self._n += 2
        while not Primes._is_prime(self._n):
            self._n += 2
        return result

In [5]:
for prime in Primes():
    print(prime)
    if prime > 100:
        break

2
3
5
7
11
13
17
19
23
29
31
37
41
43
47
53
59
61
67
71
73
79
83
89
97
101


Let's generalize that a bit so that the iterator returns only prime numbers between two given values.

In [6]:
class Primes:
    '''Iterator that returns all prime numbers (if you have enough patience, that is).
    
    Example
    -------
    for n in Primes():
        print(n)
    '''
    
    def __init__(self, low=2, high=None):
        '''constructor that takes a low and a high value, generating prime numbers greater
        than or equal to the low value, but less than the high value
        
        Parameters
        ----------
        low : int
            start value for the sequence, default value 2
        high : int
            end value for the sequence, non-inclusive, default None, i.e., the sequence continues indefinetly
            
        Raises
        ------
        ValueError
            when low is negative
        '''
        if low < 0:
            raise ValueError('lower bound should be positive')
        self._low = low
        self._high = high
        
    @staticmethod
    def _is_prime(n):
        '''verify whether a given integer is a prime
    
        Paraemeters
        -----------
        n : int
            value to check

        Returns
        -------
        bool
            True if the number is prime, False otherwise
        '''
        if n < 2:
            return False
        for divisor in range(2, int(round(math.sqrt(n), 0)) + 1):
            if n % divisor == 0:
                return False
        return True

    def __iter__(self):
        '''initializes iterator
        
        Returns
        -------
        Primes
            initialized iterator
        '''
        self._n = self._low
        while not Primes._is_prime(self._n):
            self._n += 1
        return self
    
    def __next__(self):
        '''next value of the iterator
        
        Returns
        -------
        int
            next prime number in the sequence
        '''
        result = self._n
        if self._high is not None and result >= self._high:
            raise StopIteration()
        if self._n == 2:
            self._n = 3
        else:
            self._n += 2
            while not Primes._is_prime(self._n):
                self._n += 2
        return result

We need to raise a `StopIteration` exception when the iterator should not return any more values.

In [7]:
for n in Primes(4, 18):
    print(n)

5
7
11
13
17


Note that `Primes` has the same semantics as `range`, i.e., the high value is not inclusive.

In [8]:
for n in Primes(14, 17):
    print(n)

In [9]:
try:
    for n in Primes(-3, 5):
        print(n)
except ValueError as error:
    print('error:', error, file=sys.stderr)

error: lower bound should be positive


We can easily verify that two iterators are indepedent.

In [10]:
for m in Primes(4, 18):
    for n in Primes(12, 22):
        print(f'{m:2d}, {n:2d}')

 5, 13
 5, 17
 5, 19
 7, 13
 7, 17
 7, 19
11, 13
11, 17
11, 19
13, 13
13, 17
13, 19
17, 13
17, 17
17, 19


# Generator

In [11]:
def primes():
    '''generator for prime number sequences
    
    Expample
    --------
    for n in primes():
        print(n)
    '''

    def is_prime(n):
        if n < 2:
            return False
        for divisor in range(2, int(round(math.sqrt(n), 0)) + 1):
            if n % divisor == 0:
                return False
        return True

    yield 2
    n = 3
    while True:
        if is_prime(n):
            yield n
        n += 2

In [12]:
for prime in primes():
    print(prime)
    if prime > 100:
        break

2
3
5
7
11
13
17
19
23
29
31
37
41
43
47
53
59
61
67
71
73
79
83
89
97
101


The behaviour of the generator is exactly the same as that of the iterator, but the implementation is significantly simpler.  It is straighforward to modify `primes` such that it generates only prime numbers within given bounds.

In [13]:
def primes(low=2, high=None):
    '''generator for prime number sequences
    
    Expample
    --------
    for n in primes():
        print(n)
    '''

    if low < 0:
        raise ValueError('lower bound should be positive')
        
    def is_prime(n):
        if n < 2:
            return False
        for divisor in range(2, int(round(math.sqrt(n), 0)) + 1):
            if n % divisor == 0:
                return False
        return True

    n = low
    if n <= 2:
        yield n
        n = 3
    elif n % 2 == 0:
        n += 1
    while True:
        if is_prime(n):
            if n < high:
                yield n
            else:
                break
        n += 2

We don't need to bother raising a `StopIteration` exception when the iterator should not return any more values.

In [14]:
for n in primes(4, 18):
    print(n)

5
7
11
13
17


Note that `primes` has the same semantics as `range`, i.e., the high value is not inclusive.

In [15]:
for n in primes(14, 17):
    print(n)

In [16]:
try:
    for n in primes(-3, 5):
        print(n)
except ValueError as error:
    print('error:', error, file=sys.stderr)

error: lower bound should be positive


We can easily verify that two generators are indepedent.

In [17]:
for m in primes(4, 18):
    for n in primes(12, 22):
        print(f'{m:2d}, {n:2d}')

 5, 13
 5, 17
 5, 19
 7, 13
 7, 17
 7, 19
11, 13
11, 17
11, 19
13, 13
13, 17
13, 19
17, 13
17, 17
17, 19


# itertools

`itertools` is a very nice and useful module in Python's standard libray.  It contains all we need to get our infinite sequence of prime numbers with a single line of code (except for the `is_prime` funciton.  We will also need Python's built-in `filter` function.

In [18]:
for prime in filter(is_prime, itertools.count()):
    print(prime)
    if prime > 100:
        break

2
3
5
7
11
13
17
19
23
29
31
37
41
43
47
53
59
61
67
71
73
79
83
89
97
101


We can limit the prime values taken by using a  `ragne`.

In [19]:
for prime in filter(is_prime, range(4, 18)):
    print(prime)

5
7
11
13
17


Note the rather nice and simple lambda function to restrict the prima values.

In [20]:
for m in filter(lambda x: is_prime(x), range(4, 18)):
    for n in filter(lambda x: is_prime(x), range(12, 22)):
        print(m, n)

5 13
5 17
5 19
7 13
7 17
7 19
11 13
11 17
11 19
13 13
13 17
13 19
17 13
17 17
17 19


For an application like this, `itertools.product` comes in handy.

In [21]:
for m, n in itertools.product(filter(lambda x: is_prime(x), range(4, 18)),
                              filter(lambda x: is_prime(x), range(12, 22))):
    print(m, n)

5 13
5 17
5 19
7 13
7 17
7 19
11 13
11 17
11 19
13 13
13 17
13 19
17 13
17 17
17 19
